In [1]:
import torch
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv("bonus_dataset.csv")
df.head()

,employee_id,performance,years_of_experience,projects_completed,bonus
0,EMP_001,7,2,4,124
1,EMP_002,4,1,4,82
2,EMP_003,8,7,10,178
3,EMP_004,5,7,8,138
4,EMP_005,7,8,9,170


In [3]:
df.shape

(100, 5)

In [4]:
df['performance'].values

array([ 7,  4,  8,  5,  7, 10,  3,  7,  8,  5,  4,  8,  8,  3,  6,  5,  2,
        8,  6,  2,  5,  1, 10,  6,  9,  1, 10,  3,  7,  4,  9,  3,  5,  3,
        7,  5,  9,  7,  2,  4,  9,  2, 10,  9, 10,  5,  2,  4,  7,  8,  3,
        1,  4,  2,  8,  4,  2,  6,  6, 10,  4,  6,  2, 10,  2, 10,  4,  8,
        7,  9,  8,  5,  2,  5,  8, 10,  9,  9,  1,  9,  7,  9,  8,  1,  8,
        8,  3,  1,  8,  3,  3,  1,  5, 10,  7, 10,  9,  7,  9,  8],
      dtype=int64)

Batch Gradient

In [5]:
performance = torch.tensor(df['performance'].values, dtype=torch.float32)
years_of_experience = torch.tensor(df['years_of_experience'].values, dtype=torch.float32)
projects_completed	 = torch.tensor(df['projects_completed'].values, dtype=torch.float32)
bonus = torch.tensor(df['bonus'].values, dtype=torch.float32)

In [6]:
w1 = torch.rand(1,requires_grad=True)
w2 = torch.rand(1,requires_grad=True)
w3 = torch.rand(1,requires_grad=True)
bias = torch.rand(1,requires_grad=True)

w1,w2,w3,bias

(tensor([0.7112], requires_grad=True),
 tensor([0.1135], requires_grad=True),
 tensor([0.2654], requires_grad=True),
 tensor([0.5576], requires_grad=True))

In [7]:
epochs = 5000
learning_rate = 0.005
loss_history = []

for epoch in range(epochs):
    predicted_bonus = w1*performance + w2*years_of_experience + w3*projects_completed + bias
    loss = ((predicted_bonus-bonus)**2).mean()
    loss_history.append(loss.item())
    loss.backward()
    
    with torch.no_grad():
        w1-= learning_rate*w1.grad
        w2-= learning_rate*w2.grad
        w3-= learning_rate*w3.grad
        bias-= learning_rate*bias.grad
    
    w1.grad.zero_()
    w2.grad.zero_()
    w3.grad.zero_()
    bias.grad.zero_()
    
    if epoch%100==0:
        print(f"Epoch {epoch}, loss {loss.item():0.2f}")

Epoch 0, loss 18024.38
Epoch 100, loss 18.01
Epoch 200, loss 16.07
Epoch 300, loss 14.35
Epoch 400, loss 12.81
Epoch 500, loss 11.44
Epoch 600, loss 10.21
Epoch 700, loss 9.12
Epoch 800, loss 8.14
Epoch 900, loss 7.27
Epoch 1000, loss 6.49
Epoch 1100, loss 5.79
Epoch 1200, loss 5.17
Epoch 1300, loss 4.62
Epoch 1400, loss 4.12
Epoch 1500, loss 3.68
Epoch 1600, loss 3.28
Epoch 1700, loss 2.93
Epoch 1800, loss 2.62
Epoch 1900, loss 2.34
Epoch 2000, loss 2.09
Epoch 2100, loss 1.86
Epoch 2200, loss 1.66
Epoch 2300, loss 1.48
Epoch 2400, loss 1.33
Epoch 2500, loss 1.18
Epoch 2600, loss 1.06
Epoch 2700, loss 0.94
Epoch 2800, loss 0.84
Epoch 2900, loss 0.75
Epoch 3000, loss 0.67
Epoch 3100, loss 0.60
Epoch 3200, loss 0.53
Epoch 3300, loss 0.48
Epoch 3400, loss 0.43
Epoch 3500, loss 0.38
Epoch 3600, loss 0.34
Epoch 3700, loss 0.30
Epoch 3800, loss 0.27
Epoch 3900, loss 0.24
Epoch 4000, loss 0.22
Epoch 4100, loss 0.19
Epoch 4200, loss 0.17
Epoch 4300, loss 0.15
Epoch 4400, loss 0.14
Epoch 4500, 

In [8]:
w1.item(),w2.item(),w3.item(),bias.item()

(12.047937393188477, 5.835544586181641, 2.2193942070007324, 18.928739547729492)

In [9]:
print(f"\nLearned weights and bias:")
print(f"w1: {w1.item():.2f}, w2: {w2.item():.2f}, w3: {w3.item():.2f}, bias: {bias.item():.2f}")


Learned weights and bias:
w1: 12.05, w2: 5.84, w3: 2.22, bias: 18.93


mini batch

In [16]:
w1 = torch.rand(1,requires_grad=True)
w2 = torch.rand(1,requires_grad=True)
w3 = torch.rand(1,requires_grad=True)
bias = torch.rand(1,requires_grad=True)

w1,w2,w3,bias

(tensor([0.8996], requires_grad=True),
 tensor([0.4037], requires_grad=True),
 tensor([0.1036], requires_grad=True),
 tensor([0.6401], requires_grad=True))

In [18]:
epochs = 5000
learning_rate = 0.001
loss_history = []
batch_size = 16
n_samples = len(performance)

for epoch in range(epochs):
    for i in range(0,n_samples,batch_size):
        batch_performance = performance[i :i+batch_size]
        batch_years_of_experience = years_of_experience[i:i + batch_size]
        batch_projects_completed = projects_completed[i:i + batch_size]
        batch_bonus = bonus[i:i + batch_size]
    
        predicted_bonus = w1*batch_performance + w2*batch_years_of_experience + w3*batch_projects_completed + bias
        loss = ((predicted_bonus-batch_bonus)**2).mean()
        loss_history.append(loss.item())
        loss.backward()
    
        with torch.no_grad():
            w1 -= learning_rate * w1.grad
            w2 -= learning_rate * w2.grad
            w3 -= learning_rate * w3.grad
            bias -= learning_rate * bias.grad
    
        w1.grad.zero_()
        w2.grad.zero_()
        w3.grad.zero_()
        bias.grad.zero_()
        
    if epoch%100==0:
        print(f"Epoch {epoch}, loss {loss.item():0.2f}")

Epoch 0, loss 1781.83
Epoch 100, loss 12.38
Epoch 200, loss 10.37
Epoch 300, loss 8.78
Epoch 400, loss 7.48
Epoch 500, loss 6.39
Epoch 600, loss 5.46
Epoch 700, loss 4.68
Epoch 800, loss 4.00
Epoch 900, loss 3.43
Epoch 1000, loss 2.94
Epoch 1100, loss 2.52
Epoch 1200, loss 2.16
Epoch 1300, loss 1.85
Epoch 1400, loss 1.58
Epoch 1500, loss 1.36
Epoch 1600, loss 1.16
Epoch 1700, loss 1.00
Epoch 1800, loss 0.85
Epoch 1900, loss 0.73
Epoch 2000, loss 0.63
Epoch 2100, loss 0.54
Epoch 2200, loss 0.46
Epoch 2300, loss 0.39
Epoch 2400, loss 0.34
Epoch 2500, loss 0.29
Epoch 2600, loss 0.25
Epoch 2700, loss 0.21
Epoch 2800, loss 0.18
Epoch 2900, loss 0.16
Epoch 3000, loss 0.13
Epoch 3100, loss 0.11
Epoch 3200, loss 0.10
Epoch 3300, loss 0.08
Epoch 3400, loss 0.07
Epoch 3500, loss 0.06
Epoch 3600, loss 0.05
Epoch 3700, loss 0.05
Epoch 3800, loss 0.04
Epoch 3900, loss 0.03
Epoch 4000, loss 0.03
Epoch 4100, loss 0.02
Epoch 4200, loss 0.02
Epoch 4300, loss 0.02
Epoch 4400, loss 0.02
Epoch 4500, loss 

In [19]:
print(f"Learned weights: w1 = {w1.item():.4f}, w2 = {w2.item():.4f}, w3 = {w3.item():.4f}")
print(f"Learned bias: {bias.item():.4f}")

Learned weights: w1 = 12.0189, w2 = 5.9491, w3 = 2.0710
Learned bias: 19.6182


SGD

In [20]:
w1 = torch.rand(1,requires_grad=True)
w2 = torch.rand(1,requires_grad=True)
w3 = torch.rand(1,requires_grad=True)
bias = torch.rand(1,requires_grad=True)

w1,w2,w3,bias

(tensor([0.8926], requires_grad=True),
 tensor([0.9761], requires_grad=True),
 tensor([0.2373], requires_grad=True),
 tensor([0.5285], requires_grad=True))

In [21]:
epochs = 500
learning_rate = 0.001
loss_history = []
batch_size = 16
n_samples = len(performance)

for epoch in range(epochs):
    for i in range(n_samples):
        single_performance = performance[i]
        single_years_of_experience = years_of_experience[i]
        single_projects_completed = projects_completed[i]
        single_bonus = bonus[i]
    
        predicted_bonus = w1*single_performance + w2*single_years_of_experience + w3*single_projects_completed + bias
        loss = ((predicted_bonus-batch_bonus)**2).mean()
        loss_history.append(loss.item())
        loss.backward()
    
        with torch.no_grad():
            w1 -= learning_rate * w1.grad
            w2 -= learning_rate * w2.grad
            w3 -= learning_rate * w3.grad
            bias -= learning_rate * bias.grad
    
        w1.grad.zero_()
        w2.grad.zero_()
        w3.grad.zero_()
        bias.grad.zero_()
        
    if epoch%100==0:
        print(f"Epoch {epoch}, loss {loss.item():0.2f}")

Epoch 0, loss 193.34
Epoch 100, loss 221.93
Epoch 200, loss 182.80
Epoch 300, loss 179.34
Epoch 400, loss 179.03
